In [18]:
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
import pickle
import openai


load_dotenv('credentials.env')

True

In [24]:
with open("sagemaker_documentation.pkl", "rb") as f:
    current_vectorstore = pickle.load(f)

retriever = current_vectorstore.as_retriever()

In [31]:
from langchain.prompts import PromptTemplate


prompt_template = """You are a virtual assistant for LOKA company, \
always respond with kindness and say hello when necessary,\
use the following "context" to answer the question. If it's not \
mentioned in the "context", politely respond that you don't know.

context:
"{context}"

question: "{question}"
answer:"""
QA_PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
    )


In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0613",
    temperature=1,
    request_timeout=45
)

question_chain = LLMChain(
    llm=llm,
    prompt=QA_PROMPT,
    verbose=True
)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | QA_PROMPT 
    | llm
    | StrOutputParser() 
)


In [36]:
rag_chain.invoke("What is SageMaker?")

AttributeError: 'NoneType' object has no attribute 'lower'

In [33]:
from datasets import Dataset

questions = ["What is SageMaker?", 
             "What are all AWS regions where AWS SageMaker is available?",
             "How to check if an endpoint is KMS encrypted?",
             "What are SageMaker Geospatial capabilities?"
            ]
ground_truths = [["SageMaker is a fully managed machine learning service provided by Amazon. It allows data scientists and developers to build, train, and deploy machine learning models in a production-ready environment. It also provides features like Jupyter notebook integration, Git repository association, and integration with tools such as Secrets Manager. If you need more specific information, please let me know."],
                ["AWS SageMaker is available in the following AWS Regions:- US East (Ohio)- US East (N. Virginia)- US West (N. California)- US West (Oregon)- Africa (Cape Town)- Asia Pacific (Hong Kong)- Asia Pacific (Mumbai)- Asia Pacific (Osaka)- Asia Pacific (Seoul)- Asia Pacific (Singapore)- Asia Pacific (Sydney)- Asia Pacific (Jakarta)- Asia Pacific (Tokyo)- Canada (Central)- China (Beijing)- China (Ningxia)- Europe (Frankfurt)- Europe (Ireland)- Europe (London)- Europe (Paris)- Europe (Stockholm)- Europe (Milan)- Middle East (Bahrain)- South America (São Paulo)- AWS GovCloud (US-West)"],
                ["To check if an endpoint is KMS encrypted, you can use the SAGEMAKER_ENDPOINT_CONFIGURATION_KMS_KEY_CONFIGURED rule. This rule checks whether the KmsKeyId is specified for the Amazon SageMaker endpoint configuration. If the KmsKeyId is not specified, the rule will be considered NON_COMPLIANT. You can use this rule periodically in all supported AWS regions, except China, Asia Pacific (Jakarta), Africa (Cape Town), Middle East (UAE), Asia Pacific (Hyderabad), Asia Pacific (Osaka), Asia Pacific (Melbourne), Europe (Milan), AWS GovCloud (US-East), Europe (Spain), China (Ningxia), and Europe (Zurich) Regions."],
                ["SageMaker Geospatial capabilities refer to the operations and functionalities provided by Amazon SageMaker for performing geospatial tasks. These capabilities allow users to perform operations on geospatial data and leverage machine learning algorithms for geospatial analysis"]]
answers = []
contexts = []

# Inference
for query in questions:
  answers.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

AttributeError: 'NoneType' object has no attribute 'lower'